In [1]:
from importlib import reload  
import numpy as np
import matplotlib.pyplot as plt
import pycraf
import cysgp4
from pycraf import conversions as cnv
from pycraf import protection, antenna, geometry
from astropy import units as u, constants as const
from scepter import skynet,obs
from astropy import coordinates as coord



In [2]:

###reload here
reload(skynet)
reload(obs)

### we're testing with the ie613 lofar station location
longitude = 7.9219 * u.deg
latitude = 53.0950 * u.deg
elevation = 72.0 * u.m
observatory1=cysgp4.PyObserver(longitude.value,latitude.value,elevation.to(u.km).value)
parkes=coord.EarthLocation.of_site('Parkes')
print(parkes.geodetic)
longitude = parkes.geodetic.lon
latitude = parkes.geodetic.lat
elevation = parkes.geodetic.height
observatory2=cysgp4.PyObserver(longitude.value,latitude.value,elevation.to(u.km).value)

GeodeticLocation(lon=<Longitude 148.26351001 deg>, lat=<Latitude -32.9984064 deg>, height=<Quantity 414.75974971 m>)


In [3]:



obsmode='continuum'
ras_df = protection.ra769_limits(mode=obsmode).to_pandas() # 150 MHz RAS band
ras_tab = protection.ra769_limits(mode='continuum')[7] # 1.4 GHz RAS band
# print(ras_df)
print(ras_tab)
# Rx (RAS) parameters
eta_a_rx = 100 * u.percent

frequency bandwidth T_A T_rx T_rms  P_rms_nu   Plim   Plim_nu      Slim        Slim_nu        Efield    Efield_norm 
   MHz       MHz     K   K     mK  dB(W / Hz) dB(W)  dB(W / Hz) dB(W / m2) dB(W / (Hz m2)) dB(uV2 / m2) dB(uV2 / m2)
--------- --------- --- ---- ----- ---------- ------ ---------- ---------- --------------- ------------ ------------
     1414        27  12   10 0.095     -268.8 -204.5     -278.8     -180.1          -254.4        -34.3        -48.6


### simulate telescope pointing grid

In [4]:
min_elevation = 30 * u.deg
grid_size = 3. * u.deg
npoints=5 ##pointing per cell
skygrid = skynet.pointgen(niters=5,
    step_size=grid_size,
    lat_range=(min_elevation, 90 * u.deg),
    rnd_seed=0,
    )

tel_az, tel_el, grid_info = skygrid 
# print('-' * 80)
# print('Minimum elevation: {0:.2f}'.format(min_elevation))
# print('Number of cells: {0:d}'.format(len(grid_info)))
# print('-' * 80)
# ncell=len(grid_info)
print(tel_az.shape)
## match dimensions of this simulation
# tel_az=tel_az[:,np.newaxis,np.newaxis,np.newaxis]
# tel_el=tel_el[:,np.newaxis,np.newaxis,np.newaxis]

freq = ras_tab['frequency']

p_lim = ras_tab['Plim']
pfd_lim = ras_tab['Slim']
ras_bandwidth = ras_tab['bandwidth']
margin = 0

p_tx_carrier = (-44+margin) * cnv.dBm #CISPR22 maximum emission at 150 MHz is -44 dBm
carrier_bandwidth = ras_tab['bandwidth'] #this is not specified in the french section...
duty_cycle = 100 * u.percent
print(ras_bandwidth)

# freq=10.4*u.GHz


(5, 1146)
27.0 MHz


### set up transmitter class

In [5]:
tx=obs.transmitter_info(p_tx_carrier, carrier_bandwidth, duty_cycle, d_tx=1*u.m,freq=freq)
p_tx=tx.power_tx(ras_bandwidth)
print(p_tx)

-44.0 dB(mW)


In [6]:
gain=tx.satgain1d(30*u.deg)
print(f'1d gain pattern of transmitter at phi = 30 is {gain}')

1d gain pattern of transmitter at phi = 30 is 8.337217533032337 dB


### set up receiver class

In [7]:
dish=100*u.m
eta=0.7 *100*u.percent ## in percentage
### frequency band
## transmitter parameters
sat_antenna=0.1*u.m
### observatory list will be passed on internally like this
observers = np.array([observatory1,observatory2
    ])
rx=obs.receiver_info(dish,eta,observers,freq,bandwidth=ras_bandwidth)

In [8]:
rx.location  ### observatory information is saved here

array([<PyObserver: 7.9219d, 53.0950d, 0.0720km>,
       <PyObserver: 148.2635d, -32.9984d, 0.4148km>], dtype=object)

In [9]:
###load some tle from starlink
import requests
url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle'
ctrak_starlink = requests.get(url).text

tle_list = cysgp4.tle_tuples_from_text(ctrak_starlink)

# tle_list

In [10]:
### set time and simulation settings

niters = 2
pydt = cysgp4.PyDateTime() ## take current date and time
start_times_window = 24 * u.hour
time_range, time_resol = 100*u.s, 1*u.s  # seconds
epochiters=time_range/time_resol

# print(start_times,td)
mjds = skynet.plantime(niters,start_times_window,time_range,time_resol,pydt)
print(mjds.shape)

(1, 1, 1, 2, 100, 1)


In [11]:
tles = np.array([
    cysgp4.PyTle(*tle) for tle in tle_list
    ])[:500]  # use which TLEs

print (f'we are simulating {tles.shape[0]} satellites over {niters*start_times_window.to_value(u.day)} days,')
print(f'each day/epoch contains {epochiters} iterations over with {time_range} seconds')

print(type(tles))

we are simulating 500 satellites over 2.0 days,
each day/epoch contains 100.0 iterations over with 100.0 s seconds
<class 'numpy.ndarray'>


In [12]:
simulator=obs.obs_sim(tx,rx,tles,skygrid,mjds)


In [13]:
simulator.populate()



(2, 1, 1, 1, 1, 1) (1, 1, 1, 1, 1, 500) (1, 1, 1, 2, 100, 1)
Obtaining satellite and time information, propagation for large arrays may take a while...
Done. Satellite coordinates obtained


In [14]:
satinfo=simulator.sat_info

print(simulator.topo_pos_az.shape)

(2, 1, 1, 2, 100, 500)


### calculate beam pointing in satellite reference frame compared to observatory location

In [15]:
ang_sep,delta_az,delta_el,obs_dist=obs.sat_frame_pointing(sat_info=simulator.sat_info,beam_el=20,beam_az=0)
ang_sep=simulator.txbeam_angsep(beam_az=20,beam_el=0)

print(ang_sep.shape,obs_dist.shape)

(2, 1, 1, 2, 100, 500) (2, 1, 1, 2, 100, 500)


### calculate gain in observatory direction using 1d antenna pattern

In [16]:
gtx=tx.satgain1d(ang_sep) 


### apply free space path loss over distance

In [17]:
sat_power=tx.fspl(obs_dist*u.km,gtx)

In [18]:
sat_power[0]

<Decibel [[[[[-229.53168347, -207.62972824, -208.77575334, ...,
              -210.70546472, -201.46549154, -208.55843006],
             [-229.54300146, -207.62016467, -208.78577101, ...,
              -210.70380212, -201.45310181, -208.52640761],
             [-229.55430543, -207.61060274, -208.7957821 , ...,
              -210.70213165, -201.44077496, -208.49444521],
             ...,
             [-230.56710155, -206.72154044, -209.71343795, ...,
              -210.50693668, -200.55630432, -205.73354212],
             [-230.57715066, -206.71252107, -209.72271613, ...,
              -210.50449257, -200.55000898, -205.70739864],
             [-230.58718766, -206.70351187, -209.73198585, ...,
              -210.50204024, -200.54377639, -205.68131634]],

            [[-233.01522329, -205.60632895, -212.21788989, ...,
              -209.01361134, -201.44736716, -200.50759937],
             [-233.02252137, -205.60537323, -212.22429714, ...,
              -209.00832348, -201.45973965, -200

### loop all cells

In [24]:
print(f'loop this for all {npoints} pointings')



print('running pointing grid.....')

grx=rx.antgain1d(tp_az=simulator.tel_az,tp_el=simulator.tel_el,sat_obs_az=simulator.topo_pos_az,sat_obs_el=simulator.topo_pos_el)
readout=obs.prx_cnv(sat_power,grx, outunit=u.W)
print(readout.shape)

totalprx=readout.sum(4)
prxarray=totalprx
# print(prxarray)
pwr_flux_density = (cnv.powerflux_from_prx(totalprx * 1e60, freq, 0 * cnv.dBi) * 1e-60).to(cnv.dB_W_m2)
### this step converts to ra769
# print(pwr_flux_density)

pfdarray=pwr_flux_density

print('done')

loop this for all 5 pointings
running pointing grid.....
Obtaining satellite and telescope pointing coordinates, calculation for large arrays may take a while...
Done. putting angular separation into gain pattern function


KeyboardInterrupt: 

In [ ]:
# aux_file='test.npz'

# np.savez(
#     aux_file,
#     tel_az=tel_az,
#     tel_el=tel_el,
#     grid_info=grid_info,
#     pfd_lim_W_m2=pfd_lim.to_value(u.W / u.m ** 2),
#     frequency_ghz=freq.to_value(u.GHz),
#     observatory=observatory
#     )
indexslice=0
_pfds=pfdarray[:,:,indexslice]
pfd_lin = _pfds.to_value(u.W / u.m ** 2)
pfd_avg = (np.mean(pfd_lin, axis=1) * u.W / u.m ** 2).to(cnv.dB_W_m2)
pfd_98p = (np.percentile(pfd_lin, 98., axis=0) * u.W / u.m ** 2).to(cnv.dB_W_m2)
pfd_max = (np.max(pfd_lin, axis=0) * u.W / u.m ** 2).to(cnv.dB_W_m2)

_pfd_lim_W_m2 = pfd_lim.to_value(u.W / u.m ** 2)


In [ ]:

val = pfd_avg.to_value(cnv.dB_W_m2)
skynet.plotgrid(val,grid_info,point_az=topo_pos_az[0,:,indexslice],point_el=topo_pos_el[0,:,indexslice],
                elmax=90,elmin=-90)

In [ ]:
pfdarray.shape